In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import geopandas as gpd
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import shap
import torch
import xarray as xr
from neuralhydrology.datasetzoo import get_dataset
from neuralhydrology.datautils.utils import load_scaler
from neuralhydrology.modelzoo.cudalstm import CudaLSTM
from neuralhydrology.modelzoo.customlstm import CustomLSTM
from neuralhydrology.modelzoo.ealstm import EALSTM
from neuralhydrology.utils.config import Config
from torch.utils.data import DataLoader
from tqdm.notebook import tqdm

# setting device on GPU if available, else CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)
print()

# Additional Info when using cuda
if device.type == "cuda":
    print(torch.cuda.get_device_name(0))
    print("Memory Usage:")
    print("Allocated:", round(torch.cuda.memory_allocated(0) / 1024**3, 1), "GB")
    print("Cached:   ", round(torch.cuda.memory_reserved(0) / 1024**3, 1), "GB")

Using device: cuda

NVIDIA GeForce RTX 4080 SUPER
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB


### Prepare model

In [ ]:
config_file = Path(
    "./model_runs/cudalstm_q_mm_day_mswep_no_autocorr_static_1103_191754/config.yml"
)

cudalstm_config = Config(config_file)

# create a new model instance with random weights
cuda_lstm = CudaLSTM(cfg=cudalstm_config)

# load the trained weights into the new model.
model_path = cudalstm_config.run_dir / "model_epoch024.pt"
model_weights = torch.load(
    str(model_path), map_location="cpu"
)  # load the weights from the file, creating the weight tensors on CPU
cuda_lstm.load_state_dict(
    model_weights
)  # set the new model's weights to the values loaded from file
custom_lstm = CustomLSTM(
    cfg=cudalstm_config
)  # create a new CustomLSTM (with random weights)
custom_lstm.copy_weights(cuda_lstm)  # copy the CudaLSTM weights into the CustomLSTM
# make sure we're in eval mode where dropout is deactivated
custom_lstm.eval()

# load the dataset
scaler = load_scaler(cudalstm_config.run_dir)
dataset = get_dataset(cudalstm_config, is_train=False, period="test", scaler=scaler)
dataloader = DataLoader(
    dataset, batch_size=731, shuffle=False, collate_fn=dataset.collate_fn
)

example_states = custom_lstm(next(iter(dataloader)))
example_states.keys()

### Read geometry and model data results

### Metrics by Geo Cluster

### Metrics by Hydro Cluster

### Metrics by area

### Cell states vs Evaporation

### Cell states vs Snow Depth

### Cell states vs Sub Surface Flow

### Cell states vs SWE